In [9]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [10]:
import scib
import os
import scanpy as sc
import tensorflow as tf
import matplotlib
import anndata
from scipy import sparse
import pandas as pd

## Loading and Preprocessing

In [11]:
#---Load Data---
dataset_name = 'Lung_Atlas_public'
label_key = 'cell_type'
batch_key = 'batch'


#set base path to load data: goes back one directory and then into the data
base_path = os.path.join('..', 'data')

# read dataset into an anndata object:  Category - Cells of the brain
inPath = os.path.join(base_path, f"{dataset_name}.h5ad")
adata = sc.read(inPath)

#---load model---
#set base path to load data: goes back one directory and then into the data
base_path = os.path.join('..', 'src', 'models', 'saved_models')
name = "autoencoder_final_onestep_classifier"

# load autoencoder
inPath = os.path.join(base_path, f"{name}.keras")
autoencoder = tf.keras.models.load_model(inPath)

c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\anndata\_io\specs\registry.py:281: OldFormatWarning: Element '/layers' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=iospec)
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\anndata\_io\utils.py:215: OldFormatWarning: Element '/obs/__categories/dataset' was written without encoding metadata.
  return func(*args, **kwargs)
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\anndata\_io\utils.py:215: OldFormatWarning: Element '/obs/dataset' was written without encoding metadata.
  return func(*args, **kwargs)
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\anndata\_io\utils.py:215: OldFormatWarning: Element '/obs/__categories/location' was written without encoding metadata.
  return func(*args, **kwargs)
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\anndata\_io\utils.py:215: OldFormatWarning: Element '/obs/location' was written without encoding metadata.
  return func(

In [12]:
#---autoencode data---
#adata -> numpy
INPUT = adata.X.toarray()

#autoencode data
autoencoded_data = autoencoder.predict(INPUT)

#numpy -> adata
autoencoded_data_sparse = sparse.csr_matrix(autoencoded_data)
#create anndata object
integrated = anndata.AnnData(autoencoded_data_sparse)
#Take metadata from input
integrated.obs = adata.obs
integrated.var= adata.var

1015/1015 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step


In [13]:
integrated.obsm

AxisArrays with keys: 

In [14]:
#Run PCA
scib.pp.reduce_data(
    integrated, n_top_genes=2000, batch_key="batch", pca=True, neighbors=True
)
#Cluster PCA results
scib.me.cluster_optimal_resolution(integrated, cluster_key="cluster", label_key="cell_type")

HVG
Using 45 HVGs from full intersect set
Using 84 HVGs from n_batch-1 set
Using 73 HVGs from n_batch-2 set
Using 94 HVGs from n_batch-3 set
Using 110 HVGs from n_batch-4 set
Using 136 HVGs from n_batch-5 set
Using 184 HVGs from n_batch-6 set
Using 257 HVGs from n_batch-7 set
Using 250 HVGs from n_batch-8 set
Using 392 HVGs from n_batch-9 set
Using 375 HVGs from n_batch-10 set
Using 2000 HVGs


c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\preprocessing.py:616: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  adata.var["highly_variable"] = np.in1d(adata.var_names, hvg_list)


Computed 2000 highly variable genes
PCA


c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scanpy\preprocessing\_pca\__init__.py:439: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)


Nearest Neigbours


c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cluster for cluster_0.2 with leiden


c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\clustering.py:96: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  cluster_function(adata, resolution=res, key_added=resolution_key, **kwargs)


resolution: 0.2, nmi: 0.8550294062697765
Cluster for cluster_0.4 with leiden
resolution: 0.4, nmi: 0.8309149450875044
Cluster for cluster_0.6 with leiden
resolution: 0.6, nmi: 0.8217241533944367
Cluster for cluster_0.8 with leiden
resolution: 0.8, nmi: 0.7973147512941755
Cluster for cluster_1.0 with leiden
resolution: 1.0, nmi: 0.7946207471120966
Cluster for cluster_1.2 with leiden
resolution: 1.2, nmi: 0.7858971826492686
Cluster for cluster_1.4 with leiden
resolution: 1.4, nmi: 0.7828115175738056
Cluster for cluster_1.6 with leiden
resolution: 1.6, nmi: 0.7760414597014232
Cluster for cluster_1.8 with leiden
resolution: 1.8, nmi: 0.7683854165646364
Cluster for cluster_2.0 with leiden
resolution: 2.0, nmi: 0.7548827629811241
optimised clustering against cell_type
optimal cluster resolution: 0.2
optimal score: 0.8550294062697765


(0.2, np.float64(0.8550294062697765))

## Bio Scores

In [15]:
#--Scoring- Adjusted Rand Index
ari = scib.me.ari(integrated, cluster_key="cluster", label_key="cell_type")

In [16]:
#--Scoring-- HVG overlap
scib.me.hvg_overlap(adata, integrated, batch_key=batch_key)

c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\highly_variable_genes.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hvg_dir[i.obs[batch][0]] = i.var.index[hvg["highly_variable"]]
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\highly_variable_genes.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hvg_dir[i.obs[batch][0]] = i.var.index[hvg["highly_variable"]]
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\highly_variable_genes.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always

np.float64(0.27425)

In [17]:
#--Scoring-- isolated_labels_asw
scib.me.isolated_labels_asw(integrated, label_key=label_key, batch_key=batch_key, embed="X_pca")

c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\isolated_labels.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  batch_per_lab = tmp.groupby(label_key).agg({batch_key: "count"})


isolated labels: no more than 4 batches per label
isolated labels: ['Type 1']
Type 1: 0.720223069190979


np.float32(0.72022307)

In [18]:
#--Scoring-- isolated_labels_f1
scib.me.isolated_labels_f1(integrated, label_key=label_key, batch_key=batch_key, embed="X_pca")

isolated labels: no more than 4 batches per label
isolated labels: ['Type 1']


c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\isolated_labels.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  batch_per_lab = tmp.groupby(label_key).agg({batch_key: "count"})


Cluster for iso_label_0.2 with leiden
resolution: 0.2, max_f1: 0.9807692307692307
Cluster for iso_label_0.4 with leiden
resolution: 0.4, max_f1: 0.9807692307692307
Cluster for iso_label_0.6 with leiden
resolution: 0.6, max_f1: 0.9807692307692307
Cluster for iso_label_0.8 with leiden
resolution: 0.8, max_f1: 0.9807692307692307
Cluster for iso_label_1.0 with leiden
resolution: 1.0, max_f1: 0.9807692307692307
Cluster for iso_label_1.2 with leiden
resolution: 1.2, max_f1: 0.9807692307692307
Cluster for iso_label_1.4 with leiden
resolution: 1.4, max_f1: 0.9169859514687101
Cluster for iso_label_1.6 with leiden
resolution: 1.6, max_f1: 0.9169859514687101
Cluster for iso_label_1.8 with leiden
resolution: 1.8, max_f1: 0.9169859514687101
Cluster for iso_label_2.0 with leiden
resolution: 2.0, max_f1: 0.9169859514687101
optimised clustering against cell_type
optimal cluster resolution: 0.2
optimal score: 0.9807692307692307
Type 1: 0.9807692307692307


np.float64(0.9807692307692307)

In [19]:
#--Scoring-- Normalized mutual information
scib.me.nmi(integrated, cluster_key="cluster", label_key=label_key)

np.float64(0.8550294062697765)

In [20]:
#--Scoring-- Average silhouette width (ASW)
scib.me.silhouette(integrated, label_key=label_key, embed="X_pca")

np.float32(0.6707504)

## Batch Scores

In [21]:
#--Scoring-- Graph Connectivity
scib.me.graph_connectivity(integrated, label_key=label_key)

c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scib\metrics\graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  ta

np.float64(0.8192846688120957)

In [22]:
#--Scoring-- Principal component regression score
scib.me.pcr_comparison(adata, integrated, covariate=batch_key)

c:\Users\lklei\anaconda3\envs\acip\lib\site-packages\scanpy\preprocessing\_pca\__init__.py:439: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)


np.float64(0.4042807023847575)

In [23]:
#--Scoring-- Modified average silhouette width (ASW) of batch
scib.me.silhouette_batch(integrated, batch_key=batch_key, label_key=label_key, embed="X_pca")

mean silhouette per group:                       silhouette_score
group                                 
B cell                        0.722553
Basal 1                       0.943175
Basal 2                       0.897476
Ciliated                      0.901132
Dendritic cell                0.871875
Endothelium                   0.785268
Fibroblast                    0.863059
Ionocytes                     0.879514
Lymphatic                     0.894922
Macrophage                    0.820509
Mast cell                     0.761217
Neutrophil_CD14_high          0.880672
Neutrophils_IL1R2             0.890685
Secretory                     0.865062
T/NK cell                     0.813649
Type 1                        0.852086
Type 2                        0.850254


np.float32(0.8525357)

## Aggregator function

In [ ]:
def Scoring(adata, integrated, batch_key, label_key, n_top_genes):
  
    #--Preprocessing--
    #Run PCA
    if n_top_genes == True:
        scib.pp.reduce_data(
            integrated, n_top_genes=2000, batch_key="batch", pca=True, neighbors=True
        )
    elif n_top_genes == False:
          scib.pp.reduce_data(
            integrated, batch_key="batch", pca=True, neighbors=True
        )    
          
    #Cluster PCA results
    scib.me.cluster_optimal_resolution(integrated, cluster_key="cluster", label_key=label_key)

    #--Bio Scores--

    #Adjusted Rand Index
    ari = scib.me.ari(integrated, cluster_key="cluster", label_key=label_key)

    #HVG overlap
    hvg = scib.me.hvg_overlap(adata, integrated, batch_key=batch_key)

    #isolated_labels_asw
    asw = scib.me.isolated_labels_asw(integrated, label_key=label_key, batch_key=batch_key, embed="X_pca")

    #isolated_labels_f1
    f1 = scib.me.isolated_labels_f1(integrated, label_key=label_key, batch_key=batch_key, embed="X_pca")

    #Normalized mutual information
    nmi = scib.me.nmi(integrated, cluster_key="cluster", label_key=label_key)

    #Average silhouette width (ASW)
    sil= scib.me.silhouette(integrated, label_key=label_key, embed="X_pca")

    #--Cluster Scores--

    #Graph Connectivity
    graph = scib.me.graph_connectivity(integrated, label_key=label_key)

    #Principal component regression score
    pcr = scib.me.pcr_comparison(adata, integrated, covariate=batch_key)

    # Modified average silhouette width (ASW) of batch
    sil_batch = scib.me.silhouette_batch(integrated, batch_key=batch_key, label_key=label_key, embed="X_pca")

    avg_bio = (ari+hvg+asw+f1+nmi+sil)/6
    avg_batch =(graph+pcr+sil_batch)/3

    Scores = pd.DataFrame({
        'ari': ari,
        'hvg': hvg,
        'asw': asw,
        'f1': f1,
        'nmi': nmi,
        'sil': sil,
        'graph': graph,
        'pcr': pcr,
        'sil_batch': sil_batch,
        'avg_bio': avg_bio,
        'avg_batch': avg_batch
        })
    return Scores

In [ ]:
Scores = Scoring(adata, integrated, batch_key, label_key, True)

## Testing Integrators